In [1]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np

In [81]:
def clean_mi_pl_bs(file_path, sheet_name):
    """
    Designed for cleaning PL and BS.
    Handle multi-level header.
    """
    # Read the two header rows
    header_df = pd.read_excel(
        file_path, sheet_name=sheet_name, header=None, skiprows=4, nrows=2
    )

    # Multi-level header
    top_header_raw = header_df.iloc[0]
    bottom_header = header_df.iloc[1].tolist()
    top_header = top_header_raw.ffill().tolist()

    try:
        prior_year_index = bottom_header.index("Prior_Year")
        for i in range(prior_year_index, len(top_header)):
            top_header[i] = ""
    except ValueError:
        pass

    header_tuples = [
        (str(top) if pd.notna(top) else "", str(bot) if pd.notna(bot) else "")
        for top, bot in zip(top_header, bottom_header)
    ]

    if len(header_tuples) > 1:
        header_tuples[0] = ("", "Code")
        header_tuples[1] = ("", "Description")

    multi_header = pd.MultiIndex.from_tuples(header_tuples)

    # Read main data body using original number of columns from header
    num_cols = len(header_df.columns)
    data = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=None,
        skiprows=7,
        usecols=range(num_cols),
    )

    # Combine data and headers
    data.columns = multi_header

    # Drop fully empty columns and rows
    data.dropna(how="all", axis=1, inplace=True)
    data.dropna(how="all", axis=0, inplace=True)
    data.reset_index(drop=True, inplace=True)

    # Scaling

    # Read the full sheet into a temporary DataFrame to search for the indicator
    full_sheet_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Check all cells for the presence of a scaling indicator (e.g., '000s, £'000)
    scaling_indicator_found = full_sheet_df.astype(str).apply(lambda x: x.str.contains("'000|000s", case=False)).any().any()

    if scaling_indicator_found:
        
        numeric_cols = data.select_dtypes(include=np.number).columns

        # Determine rows to scale based on the sheet name
        if sheet_name == 'PL':
            rows_to_scale = data.index[:-6]
        elif sheet_name == 'BS':
            rows_to_scale = data.index[:-1]
        else:
            # Default for any other sheets is to scale all rows
            rows_to_scale = data.index

        # Apply scaling
        if len(rows_to_scale) > 0:
            print(f"Attempting to scale {len(rows_to_scale)} rows for {sheet_name}...")
            data.loc[rows_to_scale, numeric_cols] = data.loc[rows_to_scale, numeric_cols] * 1000
        else:
            print(f"Found 0 rows to scale for {sheet_name}. Skipping multiplication.")
    else:
        print(f"No scaling indicator found in sheet '{sheet_name}'. Data will not be scaled.")

    return data

In [82]:
mi_pl = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='PL')
mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')

print("\n--- PL ---")
display(mi_pl.tail(25))

print("\n--- BS ---")
display(mi_bs.tail(25))

Attempting to scale 146 rows for PL...
Attempting to scale 133 rows for BS...

--- PL ---


C:\Users\georg\AppData\Local\Temp\ipykernel_8892\646922514.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top_header = top_header_raw.ffill().tolist()


\
      Code                                        Description   
127  Total           Unrealised gains/(losses) on investments   
128   9403  J Safra Sarasin & Property Unrealised Profit/L...   
129   9432                    Wolvercote Unrealised Gain/Loss   
130   9434                 Perceptive Unrealised gains/losses   
131   9442                           CCA Unrealised gain/loss   
132   9413           SG Unrealised Profit/Loss on Investments   
133  Total                    Foreign exchange gains/(losses)   
134   9501                                                 FX   
135  Total                    Investment expenses and charges   
136   9480                   Investment Management Fees (JSS)   
137   9481     Investment Management Fees (Kleinwort Hambros)   
138   9482                 Investment Management Fees (Pluto)   
139  Total                                          Dividends   
140   9901                                           Dividend   
141   9401               J Safra Sarasin Dividends Receivable   
142  Total                           Profit/(loss) before tax   
143      x                         Tax on ordinary activities   
144  Total               Profit/(loss) for the financial year   
145    NaN                                             Per TB   
146    NaN                                                NaN   
147    NaN                                                NaN   
148    NaN                                                NaN   
149    NaN                                                NaN   
150    NaN                                                NaN   
151    NaN  GWP of £3.8m was £3.9m below budget and suffer...   

    2025-06-01 00:00:00                                        YTD  \
                 Actual         Budget      Variance        Actual   
127       -4.288751e+05   28011.251498 -4.568863e+05  1.555048e+06   
128        2.408243e+05    6698.693442  2.341256e+05  1.369468e+06   
129       -6.696994e+05   15824.793282 -6.855242e+05  1.687664e+05   
130        0.000000e+00    2219.082836 -2.219083e+03  1.467616e+04   
131        0.000000e+00    3268.681939 -3.268682e+03  2.138270e+03   
132        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
133       -2.262223e+05       0.000000 -2.262223e+05 -1.240914e+06   
134       -2.262223e+05       0.000000 -2.262223e+05 -1.240914e+06   
135        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
136        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
137        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
138                 NaN       0.000000  0.000000e+00  0.000000e+00   
139        0.000000e+00       0.000000  0.000000e+00  2.500000e+04   
140        0.000000e+00       0.000000  0.000000e+00  2.500000e+04   
141        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
142       -2.622221e+06 -266443.125535 -2.355778e+06 -3.356821e+06   
143        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
144       -2.622221e+06 -266443.125535 -2.355778e+06 -3.356821e+06   
145        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
146        2.398342e+00       0.676727 -1.721615e+00  1.024783e+00   
147        0.000000e+00       0.000000  0.000000e+00  0.000000e+00   
148        1.529286e-01       0.167408  1.447967e-02  1.798985e-01   
149       -1.195543e-02       0.022276  3.423109e-02  1.258568e-02   
150        3.024117e-02       0.005867 -2.437420e-02  3.533784e-02   
151                 NaN            NaN           NaN           NaN   

                                                             
           Budget      Variance    Prior_Year      Variance  
127  2.047940e+05  1.350254e+06  1.377674e+06  1.773738e+05  
128  4.294579e+04  1.326522e+06  3.482387e+05  1.021229e+06  
129  1.266658e+05  4.210055e+04  1.074751e+06 -9.059842e+05  
130  1.422669e+04  4.494671e+02  1.470043e+04 -2.427000e+01  
131  2.095575e+04 -1.881748e+04 -6.001522e+0


--- BS ---


\
        Code                                        Description   
109     2767                     XoL Recoveries Control Account   
110     4162  Creditors arising out of Reinsurance Operation...   
111  Heading  Other creditors including taxation and social ...   
112     4180                    Claims Handling Costs Provision   
113     4256                                       MIB Creditor   
114     4299                                    Other Creditors   
115  Heading                       Accruals and deferred income   
116     4140  Deferred Excess of Loss Cover - Reinsurers' Sh...   
117     4141  Deferred Acquisition Costs - Reinsurers' Share QS   
118     4150                 Deferred Reinsurance Commission QS   
119     4151                 Deferred Coin ceding commission BS   
120     4152                      Deferred Profit Commission BS   
121     4153                 Coin deferred profit commission BS   
122     3120                                 Bad Debt Provision   
123     4260                                           Accruals   
124     4261                         Accruals Rescue and Excess   
125     4115                                    Trade Creditors   
126     4298                                    Legal Provision   
127      NaN                                                NaN   
128  Heading                                                NaN   
129  Heading                                  Total Liabilities   
130      NaN                                                NaN   
131      NaN                                      Liquid assets   
132      NaN                           Net Technical provisions   
133      NaN                                    Liquidity ratio   

    2025-06-01 00:00:00                                            \
                 Actual        Budget      Variance    Prior_Year   
109        1.511855e+06  0.000000e+00  1.511855e+06  0.000000e+00   
110       -9.229751e+06 -1.542472e+07  6.194969e+06 -2.021632e+07   
111       -1.168776e+06 -2.143632e+06  9.748568e+05 -2.527178e+06   
112        0.000000e+00 -1.349948e+04  1.349948e+04 -3.309552e+04   
113       -1.168776e+06 -2.130133e+06  9.613573e+05 -2.494083e+06   
114        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
115       -1.500837e+07 -2.130657e+07  6.298201e+06 -1.254362e+07   
116        2.000000e-02  2.000000e-02  0.000000e+00  8.443900e+02   
117       -8.735098e+05 -2.171860e+06  1.298350e+06 -1.850162e+06   
118       -4.476346e+06 -9.346732e+06  4.870386e+06 -5.535186e+06   
119        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
120        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
121        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
122       -7.835491e+06 -7.835491e+06  0.000000e+00 -4.064769e+06   
123       -1.428742e+06 -1.852267e+06  4.235256e+05 -7.973574e+05   
124       -2.180800e+02 -2.193900e+02  1.310000e+00 -2.931390e+03   
125        1.000000e-02  1.000000e-02  0.000000e+00  1.000000e-02   
126       -3.940625e+05 -1.000000e+05 -2.940625e+05 -2.940625e+05   
127                 NaN  0.000000e+00           NaN           NaN   
128       -2.151830e+07 -3.439433e+07  1.287602e+07 -5.340950e+07   
129       -2.811651e+08 -2.924193e+08  1.125418e+07 -3.538430e+08   
130        0.000000e+00  0.000000e+00           NaN  0.000000e+00   
131        2.429058e+07  3.815875e+07 -1.386817e+07  3.527977e+07   
132       -4.049277e+07 -3.799578e+07 -2.496994e+06 -4.632129e+07   
133        5.998746e-01  1.004289e+00 -4.044148e-01  7.616318e-01   

                                                                      \
          V to PY                                                      
109  1.511855e+06                                                NaN   
110  1.098656e+07    Creditors arising out of reinsurance operations   
111  1.358403e+06                                                NaN   
112  3.309552e+04  Other credit

In [83]:
def clean_mi_deferrals(file_path, sheet_name):

    df = pd.read_excel(file_path, sheet_name)
    df_clean = df.dropna(how='all').dropna(how='all', axis=1)

    df_clean.columns = df_clean.iloc[0]

    df_clean = df_clean[1:].reset_index(drop=True)

    return df_clean

mi_deferrals = clean_mi_deferrals(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name = 'Deferrals')

display(mi_deferrals.head())

1,Act No,Act Name,2024.0,2025.0,Grand Total
0,3524,Deferred Excess of Loss Cover c/fwd,1247667.28,4163799.79,5411467.07
1,3532,Reinsurers' Provision for Unearned Premiums - QS,4413421.05,16497285.31,20910706.36
2,4852,Provision unearned premium,-6307147.26,-23567550.47,-29874697.73
3,4152,Deferred Profit Commission BS,0.00,0.00,0


In [84]:
def clean_gross_tp_walk(file_path, sheet_name):
    """
    Clean 'Gross TP walk'
    """
    
    title = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=None,
        usecols="A",
        nrows=1
    ).iloc[0, 0]

    walk_df = pd.read_excel(
        file_path,
        sheet_name=sheet_name,
        header=0,
        skiprows=1,
        usecols="A:F"
    )

    walk_df.dropna(axis=1, how='all', inplace=True)
    walk_df.rename(columns={'Unnamed: 0': title}, inplace=True)
    
    walk_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

    walk_df.dropna(how='all', inplace=True)
    walk_df.reset_index(drop=True, inplace=True)
    
    return walk_df

def clean_unearned_premium_unexpired_risk(file_path, sheet_name):
    """
    Clean and combine three secondary tables from range H3:N20.
    """

    table_definitions = {
        "act_name": {"skiprows": 2, "nrows": 5, "usecols": "H:N"},
        "unearned_ulrs": {"skiprows": 8, "nrows": 5, "usecols": "H:N"},
        "unexpired_claims": {"skiprows": 14, "nrows": 6, "usecols": "H:N"}
    }
    
    base_headers = pd.read_excel(
        file_path, sheet_name=sheet_name, header=0, 
        skiprows=2, nrows=1, usecols="H:N"
    ).columns.tolist()

    base_headers[1] = 'Item'
    all_tables = []

    for name, region in table_definitions.items():
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=region['skiprows'] + 1,
            nrows=region['nrows'] -1,
            usecols=region['usecols']
        )
        df.columns = base_headers
        
        source_name = pd.read_excel(
            file_path, sheet_name=sheet_name, header=None, 
            skiprows=region['skiprows'], nrows=1, usecols=region['usecols']
        ).iloc[0, 1]
        
        df.insert(0, 'Source', source_name)
        all_tables.append(df)
        
    combined_df = pd.concat(all_tables, ignore_index=True)
    combined_df.dropna(how='all', axis=1, inplace=True)
    combined_df.dropna(how='all', axis=0, inplace=True)
    
    return combined_df

In [85]:
gross_tp_walk = clean_gross_tp_walk(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name = 'Sheet1')
display(gross_tp_walk)

unearned_premium_unexpired_risk = clean_unearned_premium_unexpired_risk(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name = 'Sheet1')
display(unearned_premium_unexpired_risk)

C:\Users\georg\AppData\Local\Temp\ipykernel_8892\4067572181.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  walk_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


,Gross TP walk,2025-05-01 00:00:00,2025-06-01 00:00:00,Notes
0,GAPP TPs (earned basis - includes mgmt load),2.031083e+08,2.087257e+08,Derived from mgmt accounts
1,Add:,NaN,NaN,NaN
2,Best Estimate Claims attaching to UPR,2.486052e+07,2.530988e+07,ABE Loss ratios applied to Unearned Premiums. ...
3,ENID,5.973975e+05,6.091509e+05,Annually reviewed. % of TP's
4,Run-off provision (going concern basis),4.374537e+06,4.078336e+06,Costs including MIB
5,Risk Margin,1.632948e+06,1.654699e+06,Standard formula 4%
6,NaN,3.146540e+07,3.165206e+07,NaN
7,Less:,NaN,NaN,NaN
8,Mgmt load,6.000000e+06,6.000000e+06,Mgmt load not part of S2 TP's
9,Premiums due from intermediaries not yet paid,8.590635e+06,7.775939e+06,Per accounts (cancellation provision based on ...


,Source,Act No,Item,2024,2025,Grand Total,Unnamed: 12,Comment
0,Act Name,3524.0,Deferred Excess of Loss Cover c/fwd,1.247667e+06,4.163800e+06,5.411467e+06,Deferred XoL,Per Accounts
1,Act Name,3532.0,Reinsurers' Provision for Unearned Premiums - QS,4.413421e+06,1.649729e+07,2.091071e+07,QS UPR,Per Accounts
2,Act Name,4852.0,Provision unearned premium,-6.307147e+06,-2.356755e+07,-2.987470e+07,Gross UPR,Per Accounts
3,Act Name,NaN,AURR,NaN,-3.674926e+04,-3.674926e+04,AURR,NaN
4,Unearned ULRs,3524.0,Deferred Excess of Loss Cover c/fwd,1.400000e-01,1.400000e-01,NaN,NaN,Per booking sheets
5,Unearned ULRs,3532.0,Reinsurers' Provision for Unearned Premiums - QS,7.030038e-01,7.091425e-01,NaN,NaN,Per booking sheets
6,Unearned ULRs,4852.0,Provision unearned premium,8.384163e-01,8.479929e-01,NaN,NaN,Per booking sheets
7,Unearned ULRs,NaN,AURR,NaN,1.000000e+00,NaN,NaN,NaN
8,Unexpired Claims,3524.0,Deferred Excess of Loss Cover c/fwd,8.830006e+05,3.299457e+06,4.182458e+06,NaN,NaN
9,Unexpired Claims,3532.0,Reinsurers' Provision for Unearned Premiums - QS,3.102652e+06,1.169893e+07,1.480158e+07,NaN,NaN


In [4]:
def collapse_header(header_list):
    """Collapse multiple header rows into a single header for S2 Balance Sheet and SCR."""
    transposed_headers = list(zip(*header_list))
    final_headers = []
    for col_parts in transposed_headers:
        clean_parts = [str(p) for p in col_parts if pd.notna(p) and 'Unnamed' not in str(p)]
        final_headers.append(' - '.join(clean_parts))
    return final_headers

def fill_merged_cells_in_groups(df):
    """Forward-fill the first column to handle merged cells."""
    first_col = df.columns[0]
    other_cols = df.columns[1:]
    is_separator = df[other_cols].isnull().all(axis=1)
    
    df['group'] = is_separator.cumsum()
    
    df[first_col] = df.groupby('group')[first_col].ffill()
    df.drop(columns='group', inplace=True)
    return df


def clean_s2_summary_sheet(file_path, sheet_name):
    """
    Clean S2 Balance Sheet, SCR, NL P&R risk calc, OP risk calc.
    """
    
    header_block_df = pd.read_excel(
        file_path, sheet_name=sheet_name, header=None,
        skiprows=1, nrows=2, usecols="B:F"
    )
    
    shared_headers = collapse_header(header_block_df.values.tolist())
    
    regions = {
        's2_balance_sheet': {'range': 'A2:F41', 'header_rows': 2},
        'scr': {'range': 'A43:F78', 'header_rows': 3},
        'nl_pr_risk': {'range': 'A80:F95', 'header_rows': 1},
        'op_risk': {'range': 'A98:F107', 'header_rows': 1}
    }

    cleaned_tables = {}

    for name, region in regions.items():
        
        start_row = int(region['range'].split(':')[0][1:])
        end_row = int(region['range'].split(':')[1][1:])
        
        df = pd.read_excel(
            file_path, sheet_name=sheet_name, header=None,
            usecols='B:F',
            skiprows=start_row - 1,
            nrows=(end_row - start_row + 1)
        )
        
        table_title = df.iloc[0, 0]

        if name in ['s2_balance_sheet', 'scr']:
            header_list = [list(df.iloc[i]) for i in range(region['header_rows'])]
            df.columns = collapse_header(header_list)
            df = df.iloc[region['header_rows']:].reset_index(drop=True)
            df = fill_merged_cells_in_groups(df)
            df.rename(columns={df.columns[-1]: shared_headers[-1]}, inplace=True)
        else:
            first_col_header = df.iloc[0, 0]
            
            new_headers = [first_col_header] + shared_headers[1:]
            
            df.columns = new_headers
            df = df.iloc[region['header_rows']:].reset_index(drop=True)

        df.dropna(how='all', inplace=True)
        df.dropna(how='all', axis=1, inplace=True)
        
        df.name = table_title
        cleaned_tables[name] = df

    return cleaned_tables

In [5]:
all_s2_tables = clean_s2_summary_sheet(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name='S2 Balance Sheet Summary')

for name, df in all_s2_tables.items():
    print(f"\n--- {df.name} ---")
    display(df.head(25))


--- S2 Balance Sheet ---


,S2 Balance Sheet,Item,Final - As at 30th May 2025,Final - As at 30th June 2025,Movement
0,Assets,Investments - Equities,11144956.86,11040886.4,-104070.46
1,Assets,Investments - Collective Investments,19901092.06,20894746.61,993654.55
2,Assets,Loans and Mortgages,17874691.48,17589352.8,-285338.68
3,Assets,Cash,30377167.14,30157811.97,-219355.17
4,Assets,RI Recoveries BE:,167643327.382192,170268563.704526,2625236.322334
5,Assets,RI Premium provision,18342155.303965,18984035.509822,641880.205856
6,Assets,RI Claims provision & IBNR,160786435.81,166202934.87,5416499.06
7,Assets,RI Management load,-1500000,-1500000,0
8,Assets,QS Profit Commission Adjustment,4513730.018652,3738596.44,-775133.578652
9,Assets,Creditors arising out of Reinsurance Operation...,-10672712.66,-7717896.37,2954816.29



--- SCR ---


,SCR,Item,Final - As at 30th April 2025,Final - As at 30th June 2025,Movement
0,Non-life,Premium & reserve risk,10365908.650862,10530272.232488,1.643636e+05
1,Non-life,Cat risk,559975.241914,559975.241914,0.000000e+00
2,Non-life,Lapse risk,0,0,0.000000e+00
3,Non-life,Diversification,-405999.837722,-406214.92895,-2.150912e+02
4,Non-life,Total,10519884.055055,10684032.545451,1.641485e+05
6,Counterparty,Type 1,3675997.468595,3637046.482763,-3.895099e+04
7,Counterparty,Type 2,359890.989,366670.218,6.779229e+03
8,Counterparty,Diversification,-82798.974687,-84156.890451,-1.357916e+03
9,Counterparty,Total,3953089.482909,3919559.810312,-3.352967e+04
11,Market,Interest rate,703768.605922,683872.125352,-1.989648e+04



--- NL P&R risk calc ---


,NL P&R risk calc,Final - As at 30th May 2025,Final - As at 30th June 2025,Movement
0,Previous 12 months NEP - MVL,8.146558e+06,7.752597e+06,-393960.975309
1,Previous 12 months NEP - MO,8.057036e+05,7.667404e+05,-38963.173382
2,Previous 12 months NEP - A,5.625424e+05,5.373038e+05,-25238.537506
3,Previous 12 months NEP - MFL,6.528218e+04,6.235329e+04,-2928.893802
4,Forward 12 months NEP - MVL,6.308711e+06,6.445896e+06,137184.512045
5,Forward 12 months NEP - MVL,4.604939e+05,4.705074e+05,10013.555174
6,Forward 12 months NEP - MVL,2.633791e+05,2.664356e+05,3056.469039
7,Forward 12 months NEP - MVL,4.377826e+04,4.428630e+04,508.039124
8,V(prem) - MVL,8.146558e+06,7.752597e+06,-393960.975309
9,V(prem) - MO,8.057036e+05,7.667404e+05,-38963.173382



--- OP risk calc ---


,OP risk calc,Final - As at 30th May 2025,Final - As at 30th June 2025,Movement
0,GEP - historic 1 to 12 months,79727936.81,76150932.45,-3.577004e+06
1,GEP - historic 13 - 24 months,131374797.43,125371386.41,-6.003411e+06
2,Gross TPs,198954923.028861,206279137.461737,7.324214e+06
4,Claim Op,5968647.690866,6188374.123852,2.197264e+05
5,Prem Op,54183.50112,55686.03624,1.502535e+03
6,30% of BSCR,5164158.738999,5187674.127955,2.351539e+04
7,OP risk,5164158.738999,5187674.127955,2.351539e+04
8,Op risk selection,BSCR,BSCR,1.000000e+00


In [42]:
def clean_version_control_solvency_balance_sheet (file_path, skip_rows, n_rows, element):

    '''
    Process Main SOLVENCY BALANCE SHEET
    '''

    df_headers = pd.read_excel(file_path, header=None, skiprows=3, nrows=3, usecols=range(0, 21))

    # Create combined header
    header_part1 = df_headers.iloc[0].fillna('')
    header_part2 = df_headers.iloc[1].fillna('')
    header_part3 = df_headers.iloc[2].fillna('')
    
    new_header = [f"{h1} {h2} {h3}".strip() for h1, h2, h3 in zip(header_part1, header_part2, header_part3)]

    df = pd.read_excel(file_path, header=None, skiprows=skip_rows, nrows=n_rows, usecols=range(0, 21))

    df.columns = new_header

    df = df.dropna(how='all').dropna(how='all', axis=1)

    # Forward fill in col A
    df.iloc[:, 0] = df.iloc[:, 0].ffill()
    
    # Keep "Col C BS GAAP BS Jun-25" and "Col N BS SII BS Jun-25"
    df_keep = df.iloc[:, [0, 1, 9, -1]]

    df_filtered = df_keep[df_keep.count(axis=1) != 1]  # Remove rows wih all but one NaN

    # Rename first column to `element`
    old_name = df_filtered.columns[0]
    df_filtered = df_filtered.rename(columns={old_name: element})

    return df_filtered

def clean_version_control_scr_review (file_path):
    """
    Process SCR Review 
    """
    
    df = pd.read_excel(file_path, header=None, skiprows=124, nrows=27, usecols=range(0, 22))

    df = df.dropna(how='all').dropna(how='all', axis=1)
    
    # Create combined header
    header_part1 = df.iloc[0].fillna('')
    header_part2 = df.iloc[1].fillna('')
    new_header = [f"{h1} {h2}".strip() for h1, h2 in zip(header_part1, header_part2)]
    
    # Apply header
    df = df[2:]
    df.columns = new_header
    
    # Rename the final column
    df = df.rename(columns={df.columns[-1]: "Comments"})
    
    return df.reset_index(drop=True)

def clean_version_control_s2_gaap (file_path, skip_rows):

    '''
    Process GAAP to SII and SII to GAAP
    '''

    df = pd.read_excel(file_path, header=0, skiprows=skip_rows, nrows=23, usecols=range(23, 27))

    df = df.dropna(how='all').dropna(how='all', axis=1)

    return df

def clean_version_control_s2_cap_prov_summary (file_path, skip_rows):

    '''
    Process Capital Position Summary and Actual Solvency Position vs. Planned Solvency Position
    '''

    df = pd.read_excel(file_path, header=0, skiprows=skip_rows, nrows=11, usecols=range(27, 31))

    df = df.dropna(how='all').dropna(how='all', axis=1)

    numeric_cols = df.select_dtypes(include=np.number).columns

    rows_to_exclude = df.index[[-1, -7]]

    # Get the indices of the rows to scale by dropping the excluded ones
    rows_to_scale = df.index.drop(rows_to_exclude)

    # Scale the numeric values in the selected rows
    df.loc[rows_to_scale, numeric_cols] = df.loc[rows_to_scale, numeric_cols] * 1000

    return df

In [43]:
print('\n--- Solvency Balance Sheet Assets ---')
solvency_balance_sheet_assets = clean_version_control_solvency_balance_sheet(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows = 6, n_rows = 45, element = 'Assets')
display(solvency_balance_sheet_assets)
print('\n--- Solvency Balance Sheet Liabilities ---')
solvency_balance_sheet_liabilities = clean_version_control_solvency_balance_sheet(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows = 55, n_rows = 34, element = 'Liabilities')
display(solvency_balance_sheet_liabilities)
print('\n--- Solvency Balance Sheet Capital ---')
solvency_balance_sheet_capital = clean_version_control_solvency_balance_sheet(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows = 93, n_rows = 27, element = 'Capital')
display(solvency_balance_sheet_capital)

print('\n--- SCR Review ---')
scr_review = clean_version_control_scr_review(file_path = 'documents\\2506 MICL Version Control.xlsx')
display(scr_review)

print('\n--- GAAP to SII ---')
gaap_to_s2 = clean_version_control_s2_gaap(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows=61)
display(gaap_to_s2)
print('\n--- SII to GAAP ---')
s2_to_gaap = clean_version_control_s2_gaap(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows=88)
display(s2_to_gaap)

print('\n--- Capital Position Summary ---')
june_25_solvency_2_captal_position_summary = clean_version_control_s2_cap_prov_summary(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows=6)
display(june_25_solvency_2_captal_position_summary)
print('\n--- Actual Solvency Position vs. Planned Solvency Position ---')
june_25_solvency_position_actual_vs_plan = clean_version_control_s2_cap_prov_summary(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows=21)
display(june_25_solvency_position_actual_vs_plan)


--- Solvency Balance Sheet Assets ---


,Assets,,Col C BS GAAP BS 2025-06-01 00:00:00,Col N BS SII BS 2025-06-01 00:00:00
2,Fixed Assets,Tangible,0.000000e+00,0.000000e+00
3,Fixed Assets,Intangible,0.000000e+00,0.000000e+00
4,Fixed Assets,Total,0.000000e+00,0.000000e+00
7,Investments,Bonds,2.087402e+07,2.089475e+07
8,Investments,Equity,1.104089e+07,1.104089e+07
9,Investments,Property,0.000000e+00,0.000000e+00
10,Investments,Other investments,0.000000e+00,0.000000e+00
11,Investments,Total,3.191490e+07,3.193563e+07
14,Loans,Loans to associated parties,0.000000e+00,0.000000e+00
15,Loans,Other loans,1.758935e+07,1.758935e+07



--- Solvency Balance Sheet Liabilities ---


,Liabilities,,Col C BS GAAP BS 2025-06-01 00:00:00,Col N BS SII BS 2025-06-01 00:00:00
1,Technical Provisions,RI share of deferred acquisition costs,8.735097e+05,0.000000e+00
2,Technical Provisions,Unearned premium reserves,2.991145e+07,0.000000e+00
3,Technical Provisions,Claims provisions,2.087257e+08,2.027257e+08
4,Technical Provisions,Premium provisions,0.000000e+00,2.530988e+07
5,Technical Provisions,ENID Loading,0.000000e+00,6.091509e+05
6,Technical Provisions,Intermediary receivables,0.000000e+00,-7.775939e+06
7,Technical Provisions,Run-off provision,0.000000e+00,4.078336e+06
8,Technical Provisions,Discounting of TPs (Claims),0.000000e+00,-1.657533e+07
9,Technical Provisions,Discounting of TPs (R-O Prov),0.000000e+00,-3.296990e+05
10,Technical Provisions,Discounting of TPs (Prem Prov),0.000000e+00,-1.763009e+06



--- Solvency Balance Sheet Capital ---


...

,Unnamed: 23,June - 25,May - 25,Mvmnt
0,SII OF,24917.265104,28952.219388,-4034.954284
1,Gross discounting,-18668.038575,-19395.199536,727.160961
2,RI Discounting,15129.937145,15981.167638,-851.230493
3,UPR,-29911.446990,-30935.039400,1023.592410
4,Gross Unexpired Claims,25309.878606,24860.518372,449.360235
5,RI UPR,26322.173520,27427.943290,-1105.769770
6,RI Unexpired Claims,-18984.035510,-18342.155304,-641.880206
7,Future Premiums - BBNI,0.000000,0.000000,0.000000
8,RI Default - Bad debt,70.322405,68.385536,1.936869
9,Risk margin,1654.698658,1632.948415,21.750243



--- Capital Position Summary ---


,Unnamed: 27,May - 25 v1,June - 25 v1,Mvmnt
0,GAAP OF,2.448624e+07,2.100963e+07,-3.476607e+06
1,SII OF,2.895222e+07,2.491727e+07,-4.034954e+06
2,SCR,2.330339e+07,2.340529e+07,1.019000e+05
3,Surplus to SCR,5.648829e+06,1.511975e+06,-4.136854e+06
4,SCR Ratio,1.242404e+00,1.064600e+00,-1.778040e-01
5,140% SCR Target,3.262475e+07,3.276741e+07,1.426600e+05
6,Shortfall 140% SCR,-3.672527e+06,-7.850141e+06,-4.177614e+06
7,Surplus OF above 120%,9.881511e+05,-3.169083e+06,-4.157234e+06
8,MCR,5.825848e+06,5.851323e+06,2.547500e+04
9,Surplus to MCR,2.312637e+07,1.906594e+07,-4.060429e+06



--- Actual Solvency Position vs. Planned Solvency Position ---


,Unnamed: 27,June-25,June - 25 v1,Mvmnt
0,GAAP OF,2.386900e+07,2.100963e+07,-2.859370e+06
1,SII OF,2.742200e+07,2.491727e+07,-2.504735e+06
2,SCR,2.345300e+07,2.340529e+07,-4.770977e+04
3,Surplus to SCR,3.968000e+06,1.511975e+06,-2.456025e+06
4,SCR Ratio,1.169232e+00,1.064600e+00,-1.046324e-01
5,140% SCR Target,3.283420e+07,3.276741e+07,-6.679368e+04
6,Shortfall 140% SCR,-5.412200e+06,-7.850141e+06,-2.437941e+06
7,Surplus OF above 120%,-7.216000e+05,-3.169083e+06,-2.447483e+06
8,MCR,5.863000e+06,5.851323e+06,-1.167744e+04
9,Surplus to MCR,2.155900e+07,1.906594e+07,-2.493057e+06
